Flow of this code:
1. Display stock data
2. webscrapper will read headline of different financial news - sentiment anaysis will be perfomed
3. stock data will be taken for that day - open and close
4. The sentiment score and difference values will be stores in a csv
5. The csv will be read and correlation anallysis will be performed

In [1]:
# import required libraries
import numpy as np
import pandas as pd
import yfinance as yf
import plotly.graph_objs as go
import requests
from bs4 import BeautifulSoup as bs
import re
import json
from pandas_datareader import data as pdr
from datetime import datetime, timedelta

In [2]:
#temporary graphs1
def get_stock_price_data(target):
  data = yf.download(tickers=target, period='5d', interval='5m')
  return data

targets = ['GS', 'MS', 'JPM', 'BAC', 'COF']
stock_data = get_stock_price_data(targets[1])

fig = go.Figure(data=[go.Candlestick(x=stock_data.index,
                open=stock_data['Open'],
                high=stock_data['High'],
                low=stock_data['Low'],
                close=stock_data['Close'])])

fig.update_layout(title=f'Candlestick Chart for {targets[1]} Stock',
                  xaxis_title='Date',
                  yaxis_title='Price')

fig.show()

[*********************100%%**********************]  1 of 1 completed


In [3]:
csv_filename = 'stock_news_data.csv'

get difference returns the difference between open and close of the stock data on that day

In [4]:
def get_difference(symbol):
  today = datetime.today().date()
  one_day_before = today - timedelta(days=1)
  one_day_after = today + timedelta(days=1)
  yf.pdr_override()
  # Download dataframe
  #get date from news articles
  data = pdr.get_data_yahoo(symbol, start=one_day_before, end=one_day_after)
  diff = 0
  # Check if data is available for the selected date range
  if not data.empty:
      open_price = data['Open'][0]
      close_price = data['Close'][0]
      # sentiment_val = sentiment_val
      # data = {'Difference':[close_price-open_price], 'Sentiment_score':[sentiment_val]}
      print(f"Open Price: {open_price}")
      print(f"Close Price: {close_price}")
      diff = close_price-open_price
      return diff


this function writes the sentiment value and diff to a csv

In [5]:
def write_to_csv(sentiment_score, diff, h, a):
  data = {'Difference':[diff], 'Sentiment_score':[sentiment_score], 'Headline':h, 'Attention':a}
  df = pd.DataFrame(data)
  df.to_csv(csv_filename, mode='a', header=False, index=False)

In [6]:
!pip install torchtext==0.6.0
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.3 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.15.2
    Uninstalling torchtext-0.15.2:
      Successfully uninstalled torchtext-0.15.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.0 MB/s eta 0:00:00


init_token: This token represents the beginning of a sequence. It's a special token that is added to the start of each input sequence before tokenization. In the context of BERT, this token is used to signal the model that a new sequence is beginning.

eos_token: This token stands for "end of sequence". Similar to init_token, it's a special token added to the end of each input sequence before tokenization. It indicates to the model that the sequence has ended.

pad_token: This token is used for padding sequences to ensure that all sequences within a batch have the same length. In BERT, sequences are often batched together for processing, and sequences within a batch need to have consistent lengths. Padding tokens are added to the end of shorter sequences to match the length of the longest sequence in the batch.

unk_token: This token represents an "unknown" or out-of-vocabulary (OOV) token. If a word in the input text is not in the model's vocabulary, it will be replaced with the unk_token.

In the code, these special tokens are used for various purposes:

init_token_idx, eos_token_idx, pad_token_idx, and unk_token_idx: These variables store the token IDs (integer representations) of the special tokens. Token IDs are used to identify tokens in BERT's vocabulary.

tokenize_and_cut(): This function takes a sentence as input and tokenizes it using the BERT tokenizer. It also cuts the token list to ensure the sequence doesn't exceed the maximum input length allowed by the tokenizer. init_token and eos_token are added at the beginning and end of the sequence, respectively, within this function.

TEXT Field: The TEXT field is defined with specific attributes including init_token, eos_token, pad_token, and unk_token. These attributes specify how the special tokens should be used during preprocessing and tokenization of text data.

LABEL Field: The LABEL field is defined to handle labels for classification tasks. It's separate from the text data and doesn't involve special tokens.

In [7]:
# Import necessary libraries and modules
import torch
from torchtext import data, datasets
from torchtext.vocab import Vocab
import random
import numpy as np
from transformers import BertTokenizer, BertModel

'''
BERT models require text inputs to be converted into sequences of integers (token IDs) that represent words, subwords, or special tokens.
Special tokens such as [CLS], [SEP], [PAD], and [UNK] are not part of the model's vocabulary but are essential for its functioning.
They need to be converted to their respective token IDs so that they can be used as inputs
'''


# Initialize a BERT tokenizer
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define special tokens for BERT
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

# Print the special tokens
print(init_token, eos_token, pad_token, unk_token)

# Convert special tokens to their corresponding token IDs
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

# Print the token IDs for the special tokens
print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

# Get the maximum input length allowed by the BERT tokenizer
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']

# Tokenization and cutting function
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    return tokens

# Set a random seed for reproducibility
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
'''
In PyTorch, torch.backends.cudnn.deterministic is a setting that controls the determinism of some CUDA (GPU) operations.
When set to True, it ensures that CUDA operations are deterministic, meaning that they will produce the same results across runs,
given the same input and model weights. This setting is often used to enhance reproducibility in deep learning experiments,
particularly when running code on GPUs.
'''
torch.backends.cudnn.deterministic = True

'''
batch_first = True:
This parameter specifies that the first dimension of the tensors (i.e., the batch dimension) should be the batch size. This is important because many deep learning frameworks, including PyTorch, expect the batch dimension to be the first dimension of input tensors.

use_vocab = False:
This parameter indicates that the field should not build a vocabulary.
In this case, you are directly converting the input text into token IDs using the tokenizer.convert_tokens_to_ids function.
Therefore, there is no need to build a vocabulary of words or subwords. You're essentially using the BERT tokenizer's vocabulary.
'''

# Define the TEXT field for preprocessing
TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

# Define the LABEL field
LABEL = data.LabelField(dtype = torch.float)

# Load and split the IMDB dataset
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

# Build the vocabulary for the LABEL field
LABEL.build_vocab(train_data)
print(LABEL.vocab.stoi)

# Set batch size and device (GPU if available, else CPU)
BATCH_SIZE = 128
test_data = test_data
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create iterators for training, validation, and testing datasets
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device)

# Load the BERT model
bert = BertModel.from_pretrained('bert-base-uncased')
#5 min 2 secs


[CLS] [SEP] [PAD] [UNK]
101 102 0 100
downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:02<00:00, 32.7MB/s]


defaultdict(None, {'neg': 0, 'pos': 1})


In [9]:
# Import necessary libraries
import torch.nn as nn
import torch.optim as optim

'''
self.bert = bert: Stores the pre-trained BERT model as a member of the custom model.
self.rnn = nn.GRU(...): Initializes a GRU layer with specified parameters. The GRU processes the input embeddings and captures sequential dependencies.
self.out = nn.Linear(...): Initializes an output linear layer. The number of units depends on whether the GRU is bidirectional.
self.dropout = nn.Dropout(...): Initializes a dropout layer to prevent overfitting.
'''

'''
GRU, or Gated Recurrent Unit, is a type of recurrent neural network (RNN) architecture that is designed to capture and process sequential data,
 such as sequences of text, time series data, and more. GRU is an improvement over the traditional vanilla RNN and Long Short-Term Memory (LSTM) units.
 It was introduced as a more computationally efficient alternative to LSTMs while still being capable of capturing long-range dependencies in sequences.
'''
# Define a custom sentiment analysis model combining BERT and GRU
class BERTGRUSentiment(nn.Module):
    def __init__(self, bert, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        super().__init__()

        # Initialize BERT as a member of the model
        self.bert = bert
        embedding_dim = bert.config.to_dict()['hidden_size']

        # Initialize the GRU layer with specified parameters
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers=n_layers,
                          bidirectional=bidirectional,
                          batch_first=True,
                          dropout=0 if n_layers < 2 else dropout)

        # Initialize the output linear layer
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)

        # Initialize a dropout layer
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):
        # Pass the input text through BERT
        with torch.no_grad():
            embedded = self.bert(text)[0]

        # Pass the BERT embeddings through the GRU layer
        _, hidden = self.rnn(embedded)

        # Concatenate the hidden states if bidirectional, else use the last hidden state
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1))
        else:
            hidden = self.dropout(hidden[-1, :, :])

        # Pass the hidden state through the output linear layer
        output = self.out(hidden)

        return output
'''
forward method of this custom model processes the input text through BERT,
feeds the BERT embeddings through a GRU layer, applies dropout, and then passes the processed hidden state through an output
linear layer to make a sentiment prediction. The model's architecture combines BERT's contextual embeddings with a GRU layer to capture
sequential information and make sentiment predictions
'''
# Define hyperparameters
BIDIRECTIONAL = True
N_LAYERS = 2
HIDDEN_DIM = 256
DROPOUT = 0.25
OUTPUT_DIM = 1

# Create an instance of the sentiment analysis model
model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

# Count the number of trainable parameters in the model
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of trainable parameters: {trainable_params}')

# Freeze BERT parameters to prevent them from being updated during training
for name, param in model.named_parameters():
    if name.startswith('bert'):
        param.requires_grad = False

# Count the remaining trainable parameters after freezing BERT
remaining_trainable_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of remaining trainable parameters: {remaining_trainable_parameters}')

# Initialize the optimizer for parameter updates during training
optimizer = optim.Adam(model.parameters())

# Define the loss criterion for the binary classification task
criterion = nn.BCEWithLogitsLoss()

# Move the model and criterion to the specified device (CPU or GPU)
model = model.to(device)
criterion = criterion.to(device)

# Define a function to calculate binary accuracy
def binary_accuracy(preds, y):
    preds = preds > 0.5
    acc = (preds == y).sum().item() / y.size(0)
    return acc

# Define the training function
def train(model, iterator, optimizer, criterion):
    # Set the model in training mode
    model.train()
    epoch_loss = 0
    epoch_acc = 0

    for step, batch in enumerate(iterator):
        # Clear the gradients of the optimizer
        optimizer.zero_grad()

        # Make predictions using the model
        preds = model(batch.text)

        # Remove the second dimension of the predictions
        preds = preds.squeeze(dim=1)

        # Compute the loss
        loss = criterion(preds, batch.label)

        # Compute the binary accuracy
        acc = binary_accuracy(preds, batch.label)

        # Compute gradients and update model parameters
        loss.backward()
        optimizer.step()

        # Accumulate loss and accuracy for the epoch
        epoch_loss += loss.item()
        epoch_acc += acc

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

# Define the evaluation function
def evaluate(model, iterator, criterion):
    # Set the model in evaluation mode
    model.eval()
    epoch_loss = 0
    epoch_acc = 0

    # Disable gradient computation during evaluation
    with torch.no_grad():
        for step, batch in enumerate(iterator):
            # Make predictions using the model
            preds = model(batch.text)

            # Remove the second dimension of the predictions
            preds = preds.squeeze(dim=1)

            # Compute the loss
            loss = criterion(preds, batch.label)

            # Compute the binary accuracy
            acc = binary_accuracy(preds, batch.label)

            # Accumulate loss and accuracy for the epoch
            epoch_loss += loss.item()
            epoch_acc += acc

    return epoch_loss / len(iterator), epoch_acc / len(iterator)


Number of trainable parameters: 112241409
Number of remaining trainable parameters: 2759169


In [11]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

N_EPOCHS = 2

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):


    start_time =  time.time()

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model.pt')

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Training in process....
Training in process....
Training in process....
Training in process....
Training in process....
Training in process....
Evaluating....
Evaluating....
Epoch: 01 | Epoch Time: 13m 47s
	Train Loss: 0.474 | Train Acc: 75.52%
	 Val. Loss: 0.285 |  Val. Acc: 89.37%
Training in process....
Training in process....
Training in process....
Training in process....
Training in process....
Training in process....
Evaluating....
Evaluating....
Epoch: 02 | Epoch Time: 13m 53s
	Train Loss: 0.271 | Train Acc: 88.58%
	 Val. Loss: 0.263 |  Val. Acc: 90.51%


In [12]:
model.load_state_dict(torch.load('model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Evaluating....
Evaluating....
Evaluating....
Evaluating....
Evaluating....
Evaluating....
Evaluating....
Evaluating....
Evaluating....
Test Loss: 0.255 | Test Acc: 90.80%


1. Evaluation Mode:
When a model is set to evaluation mode (using model.eval()), it tells the model to behave in a way that is suitable for evaluating its performance on unseen data. In this mode:

Dropout layers, which are commonly used during training to prevent overfitting, are turned off. This is important because you don't want randomness in the model's predictions during evaluation, which could lead to inconsistent results.
Batch normalization layers use the population statistics instead of mini-batch statistics. This helps ensure that the evaluation results are consistent regardless of the batch size.
Certain layers like GRU or LSTM may have different behaviors during evaluation due to their internal states.
2. Train Mode:
When a model is set to train mode (using model.train()), it prepares the model for training on a training dataset. In this mode:

Dropout layers are turned on, which introduces stochasticity in order to prevent overfitting by reducing the reliance on specific neurons.
Batch normalization layers use mini-batch statistics to normalize activations and improve training convergence.
Layers like GRU or LSTM also have their internal states configured for gradient computation during backpropagation.
3. Other Modes:
While the most common modes are "evaluation mode" and "train mode," some models might have additional modes or behaviors. These could include:

Inference Mode: A specialized mode where the model focuses on producing predictions without any intention to learn or adapt. This can be useful when deploying the model for production.
Fine-Tuning Mode: In transfer learning scenarios, the model might have a mode specific to fine-tuning certain layers while keeping others frozen. This allows the model to adapt to new tasks while retaining the learned features from pre-training.

In [13]:
def predict_sentiment(model, tokenizer, sentence):
    # Set the model in evaluation mode
    model.eval()

    # Tokenize the input sentence
    tokens = tokenizer.tokenize(sentence)

    # Limit the token length to avoid exceeding BERT's maximum input length
    tokens = tokens[:max_input_length - 2]

    # Convert tokens to their corresponding token IDs and add special tokens
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]

    # Convert the indexed tokens to a tensor and move it to the specified device
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)  # Add a batch dimension

    # Make a prediction using the model
    prediction = torch.sigmoid(model(tensor))

    # You can also include the following lines to get attention scores from the model
    # outputs = model(tensor, output_attentions=True)
    # attention_scores = outputs.attentions
    # ret_arr = [prediction.item(), attention_scores]

    # Return the predicted sentiment as a probability
    return prediction.item()


In [15]:
def get_news_one(target):
  url = f'https://www.google.com/finance/quote/{target}:NYSE'
  r = requests.get(url)
  s = bs(r.text, 'html.parser')
  # print(s)
  arrays = re.findall(r'\[\s*.*?\s*\]', str(s))

  # Parse the arrays into separate lists or objects
  array_values = []
  for array in arrays:
      try:
          array_values.append(json.loads(array))
      except json.JSONDecodeError:
          # Handle invalid JSON format if needed
          pass
  news_list = []
  news_elements = s.find_all('div', {'class': 'Yfwt5'})

  for news_element in news_elements:
      # Extract the news title
      title = news_element.text.strip()


      # Find the parent <a> tag and extract the 'href' attribute
      parent_a_tag = news_element.find_parent('a')
      link = parent_a_tag['href']

      news_list.append({'title': title, 'link': link})
  for i in news_list:
    print(i['title'])
    print(i['link'])
    diff = get_difference(target)
    print(diff)
    a = predict_sentiment(model, tokenizer, i['title'])
    print(a)
    # inputs = tokenizer(i['title'], return_tensors='pt', padding=True, truncation=True)
    # outputs = model(**inputs)
    attention_scores = 0
    # write_to_csv(a, diff, i['title'], outputs.attentions)
    write_to_csv(a, diff, i['title'], attention_scores)

In [16]:
targets = ['GS', 'MS', 'JPM', 'BAC', 'COF']
for i in targets:
  get_news_one(i)

Nvidia may be the AI stock for now, but here are the picks for later, says 
Goldman Sachs
https://www.marketwatch.com/story/nvidia-may-be-the-ai-stock-for-now-but-here-are-the-picks-for-later-says-goldman-sachs-b84b5818
[*********************100%%**********************]  1 of 1 completed
Open Price: 322.4800109863281
Close Price: 318.7900085449219
-3.69000244140625
0.6603665947914124
Bank fined £5.4m after energy traders used WhatsApp
https://www.bbc.com/news/business-66590824
[*********************100%%**********************]  1 of 1 completed
Open Price: 84.70999908447266
Close Price: 82.9800033569336
-1.7299957275390625
0.5565543174743652
Apple is now the most under-owned large-cap U.S. tech stock - Morgan 
Stanley By Investing.com
https://www.investing.com/news/stock-market-news/apple-is-now-the-most-underowned-largecap-us-tech-stock--morgan-stanley-432SI-3159646
[*********************100%%**********************]  1 of 1 completed
Open Price: 84.70999908447266
Close Price: 82.98000

In [17]:
from google.colab import files
import pandas as pd

files.download('stock_news_data.csv')


# Load the CSV file into a DataFrame
csv_filename = 'stock_news_data.csv'
df = pd.read_csv(csv_filename)

# Select the two columns by index (0-based)
column1_idx = 0
column2_idx = 1

# Extract the selected columns
column1 = df.iloc[:, column1_idx].astype(float)
column2 = df.iloc[:, column2_idx].astype(float)

# Calculate the correlation between the two columns
correlation = column1.corr(column2)

print(f'Correlation: {correlation:.2f}')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Correlation: -0.16


To improve resultd:
1. Add more stocks synbols
2. Increase epochs

In summary, a correlation coefficient of -0.31 between the close-open price difference of stock data and the sentiment score of the headline suggests a moderate negative relationship between these variables. This relationship could be indicative of how negative sentiment in news headlines might be associated with smaller intraday stock price movements